In [6]:
import pickle
import pandas as pd
import numpy as np
from scipy.sparse import hstack


In [7]:
with open("../models/mental_health_classifier.pkl", "rb") as f:
    model = pickle.load(f)

with open("../models/tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)

with open("../models/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

with open("../models/emotion_encoder.pkl", "rb") as f:
    emotion_encoder = pickle.load(f)


In [8]:
def assign_severity(predicted_label, confidence):
    """
    Maps mental health prediction to severity level.
    """
    if predicted_label == "Suicidal":
        return "High"

    elif predicted_label in ["Depression", "Bipolar"]:
        if confidence >= 0.70:
            return "High"
        else:
            return "Medium"

    elif predicted_label in ["Anxiety", "Stress"]:
        if confidence >= 0.60:
            return "Medium"
        else:
            return "Low"

    elif predicted_label == "Normal":
        return "Low"

    else:
        return "Medium"


In [9]:
sample_text = "I feel extremely hopeless and I don't want to live anymore."
sample_emotion = "sadness"

# Vectorize text
X_text = tfidf.transform([sample_text])

# Encode emotion
emotion_encoded = emotion_encoder.transform([sample_emotion]).reshape(-1, 1)

# Combine features
X_final = hstack([X_text, emotion_encoded])

# Predict
pred_class = model.predict(X_final)[0]
confidence = np.max(model.predict_proba(X_final))
predicted_label = label_encoder.inverse_transform([pred_class])[0]

# Assign severity
severity = assign_severity(predicted_label, confidence)

predicted_label, confidence, severity


('Suicidal', np.float64(0.4741433271400671), 'High')

In [10]:
test_cases = [
    ("Normal", 0.9),
    ("Anxiety", 0.65),
    ("Stress", 0.55),
    ("Depression", 0.75),
    ("Suicidal", 0.40)
]

for label, conf in test_cases:
    print(label, conf, "→", assign_severity(label, conf))


Normal 0.9 → Low
Anxiety 0.65 → Medium
Stress 0.55 → Low
Depression 0.75 → High
Suicidal 0.4 → High
